In [ ]:
#### The code is from Irene Yi-Ju Su ####
from google.colab import drive
drive.mount ("/content/drive/")
import os
os.chdir ("/content/drive/MyDrive/TEST")

Mounted at /content/drive/


In [ ]:
import pandas as pd
import re
df_QA = pd.read_csv("QA_pairs.csv")
output_max_length = 1024

In [ ]:
def clean_text(text):
  if pd.isna(text): return ""
  if not isinstance(text, str): text = str(text)
  return re.sub(r'\s+', " ", text.lower().strip())
def df_prepare_dataset(df):
  df["Question"], df["Dataset_Answer"] = df["Question"].apply(clean_text), df["Dataset_Answer"].apply(clean_text)
  df = df[(df["Question"] !="") & (df["Dataset_Answer"] !="")]
  return df
def clean_generated(generated):
  temp = re.sub(r'[\n|\\|\x0f-\x1f|\x7f-\xff]', "", generated)
  temp = re.sub(r'\s+', " ", temp).capitalize()
  temp = re.sub(r'\.{4,}', '...', temp)
  temp = re.sub(r'\.([a-zA-Z])', r'. \1', temp)
  temp = re.sub(r'(?<=[\.\?\!]\s)(\w)|^(\w)', lambda m: m.group().upper(), temp)
  return temp

In [ ]:
def get_answer(question, model):
    device = next(model.parameters()).device
    input = tokenizer(
        f"question: {clean_text(question)} answer: ", #prompt
        return_tensors="pt",  #no need to pad. If pad make additional pairs, which result in worse result, waste memory, waste time
    ).to(device)
    output = model.generate(
        **input,
        do_sample = True, temperature = 0.7, top_p = 0.9,
        max_length = output_max_length, num_return_sequences = 1, no_repeat_ngram_size = 2, #no need to pad. Let model stop natually or at max_length
    )
    generated = tokenizer.decode(output[0], skip_special_tokens=True).split("answer:")[-1].strip() #only get text after "question: [question body] answer:"
    return clean_generated(generated)

In [ ]:
from transformers import AutoModelForCausalLM, GPT2TokenizerFast
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
Baseline_GPT2=[]
for i in range(100):
    Baseline_GPT2.append(get_answer(df_QA.iloc[i]["Question"], model))
df_QA["GPT2_Answer"]=Baseline_GPT2

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/TEST/MODEL/train_gpt2")
tokenizer = GPT2TokenizerFast.from_pretrained("/content/drive/MyDrive/TEST/MODEL/train_gpt2")
tokenizer.pad_token = tokenizer.eos_token
Finetuned_GPT2=[]
for i in range(100):
    Finetuned_GPT2.append(get_answer(df_QA.iloc[i]["Question"], model))
df_QA["GPT2_Finetuned"]=Finetuned_GPT2

In [ ]:
df_QA.to_csv("QA_pairs.csv",index = False)